In [2]:
!pip install pyfolio
import sys
import pyfolio


import pandas as pd

# test_returns = 


# with pyfolio.plotting.plotting_context(context = "paper",font_scale=1.1):
#         pyfolio.create_full_tear_sheet(
#             returns=test_returns, benchmark_rets=baseline_returns, set_context=False
#         )


You should consider upgrading via the '/Users/nm/.pyenv/versions/3.7.3/bin/python3.7 -m pip install --upgrade pip' command.


ImportError: No module named 'pyfolio'